# Lesson I4: Error Handling

**Duration**: 120-135 minutes  
**Stage**: Intermediate (Building Skills)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Design and implement custom error types using enums and structs
2. Use the `?` operator effectively for error propagation
3. Handle multiple error types in a single function using error conversion
4. Apply error handling best practices for robust applications
5. Choose between `panic!`, `Result`, and `Option` based on context

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. What's the difference between `Option<T>` and `Result<T, E>`?
2. How do you handle a `Result` using pattern matching?
3. What does the `?` operator do?
4. How do you define custom enums with data?

**Answers**: 1) `Option` for presence/absence, `Result` for success/failure with error info, 2) `match result { Ok(val) => ..., Err(e) => ... }`, 3) Early return on error, unwrap on success, 4) `enum Name { Variant(Type), ... }`

---

## 🧠 Key Concepts

### Error Handling Philosophy

Rust's approach to error handling:
- **Explicit**: Errors are part of the type system
- **Recoverable vs Unrecoverable**: `Result` vs `panic!`
- **Zero-cost**: No runtime overhead for error handling
- **Composable**: Errors can be chained and transformed

### Error Types Hierarchy

1. **Recoverable Errors**: Use `Result<T, E>`
   - File not found
   - Network timeout
   - Invalid user input

2. **Unrecoverable Errors**: Use `panic!`
   - Array bounds violation
   - Assertion failures
   - Programming logic errors

### Custom Error Design Patterns

- **Enum-based**: Multiple error variants
- **Struct-based**: Rich error context
- **Trait-based**: Error conversion and display
- **Library integration**: Working with external error types

---

## 🔬 Live Code Exploration

### Custom Error Types

In [ ]:
use std::fmt;

// Simple enum-based error type
#[derive(Debug, Clone)]
enum MathError {
    DivisionByZero,
    NegativeSquareRoot,
    InvalidInput(String),
    Overflow,
}

// Implement Display for user-friendly error messages
impl fmt::Display for MathError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            MathError::DivisionByZero => write!(f, "Cannot divide by zero"),
            MathError::NegativeSquareRoot => write!(f, "Cannot take square root of negative number"),
            MathError::InvalidInput(msg) => write!(f, "Invalid input: {}", msg),
            MathError::Overflow => write!(f, "Mathematical overflow occurred"),
        }
    }
}

// Implement std::error::Error trait
impl std::error::Error for MathError {}

// Math operations with error handling
fn safe_divide(a: f64, b: f64) -> Result<f64, MathError> {
    if b == 0.0 {
        Err(MathError::DivisionByZero)
    } else if a.is_infinite() || b.is_infinite() {
        Err(MathError::Overflow)
    } else {
        Ok(a / b)
    }
}

fn safe_sqrt(x: f64) -> Result<f64, MathError> {
    if x < 0.0 {
        Err(MathError::NegativeSquareRoot)
    } else if x.is_infinite() {
        Err(MathError::Overflow)
    } else {
        Ok(x.sqrt())
    }
}

fn parse_and_sqrt(input: &str) -> Result<f64, MathError> {
    let number: f64 = input.parse()
        .map_err(|_| MathError::InvalidInput(format!("'{}' is not a valid number", input)))?;
    
    safe_sqrt(number)
}

fn custom_error_demo() {
    println!("=== Custom Error Types Demo ===");
    
    let test_cases = vec![
        ("25", "Valid input"),
        ("-4", "Negative number"),
        ("abc", "Invalid format"),
        ("16", "Valid input"),
    ];
    
    for (input, description) in test_cases {
        print!("Testing '{}' ({}): ", input, description);
        
        match parse_and_sqrt(input) {
            Ok(result) => println!("✅ √{} = {:.2}", input, result),
            Err(e) => println!("❌ Error: {}", e),
        }
    }
    
    // Division examples
    println!("\n=== Division Examples ===");
    let divisions = vec![(10.0, 2.0), (5.0, 0.0), (f64::INFINITY, 2.0)];
    
    for (a, b) in divisions {
        match safe_divide(a, b) {
            Ok(result) => println!("✅ {} ÷ {} = {:.2}", a, b, result),
            Err(e) => println!("❌ {} ÷ {} failed: {}", a, b, e),
        }
    }
}

custom_error_demo();

### Error Propagation and the `?` Operator

In [ ]:
// Complex calculation that can fail at multiple steps
fn complex_calculation(a: f64, b: f64, c: f64) -> Result<f64, MathError> {
    // Step 1: Calculate discriminant
    let discriminant = b * b - 4.0 * a * c;
    
    // Step 2: Take square root (can fail)
    let sqrt_discriminant = safe_sqrt(discriminant)?;
    
    // Step 3: Calculate numerator
    let numerator = -b + sqrt_discriminant;
    
    // Step 4: Calculate denominator
    let denominator = 2.0 * a;
    
    // Step 5: Divide (can fail)
    let result = safe_divide(numerator, denominator)?;
    
    Ok(result)
}

// Alternative without ? operator (more verbose)
fn complex_calculation_verbose(a: f64, b: f64, c: f64) -> Result<f64, MathError> {
    let discriminant = b * b - 4.0 * a * c;
    
    let sqrt_discriminant = match safe_sqrt(discriminant) {
        Ok(val) => val,
        Err(e) => return Err(e),
    };
    
    let numerator = -b + sqrt_discriminant;
    let denominator = 2.0 * a;
    
    let result = match safe_divide(numerator, denominator) {
        Ok(val) => val,
        Err(e) => return Err(e),
    };
    
    Ok(result)
}

// Chain multiple operations
fn process_numbers(inputs: Vec<&str>) -> Result<Vec<f64>, MathError> {
    let mut results = Vec::new();
    
    for input in inputs {
        let number = input.parse::<f64>()
            .map_err(|_| MathError::InvalidInput(format!("Invalid number: {}", input)))?;
        
        let sqrt_result = safe_sqrt(number)?;
        results.push(sqrt_result);
    }
    
    Ok(results)
}

fn error_propagation_demo() {
    println!("=== Error Propagation Demo ===");
    
    // Quadratic formula examples
    let equations = vec![
        (1.0, -5.0, 6.0, "x² - 5x + 6 = 0"),
        (1.0, 2.0, 5.0, "x² + 2x + 5 = 0 (negative discriminant)"),
        (0.0, 1.0, 1.0, "0x² + x + 1 = 0 (division by zero)"),
    ];
    
    for (a, b, c, description) in equations {
        println!("\nSolving: {}", description);
        
        match complex_calculation(a, b, c) {
            Ok(root) => println!("✅ One root: {:.2}", root),
            Err(e) => println!("❌ Error: {}", e),
        }
    }
    
    // Processing multiple inputs
    println!("\n=== Processing Multiple Inputs ===");
    
    let test_inputs = vec![
        vec!["4", "9", "16", "25"],
        vec!["1", "4", "-9", "16"],  // Contains negative number
        vec!["1", "abc", "16"],       // Contains invalid input
    ];
    
    for (i, inputs) in test_inputs.iter().enumerate() {
        println!("\nTest set {}: {:?}", i + 1, inputs);
        
        match process_numbers(inputs.clone()) {
            Ok(results) => {
                println!("✅ Square roots: {:?}", 
                        results.iter().map(|x| format!("{:.2}", x)).collect::<Vec<_>>());
            },
            Err(e) => println!("❌ Processing failed: {}", e),
        }
    }
}

error_propagation_demo();

### Multiple Error Types and Conversion

In [ ]:
use std::num::ParseIntError;
use std::io;

// Application-specific error that can wrap multiple error types
#[derive(Debug)]
enum AppError {
    Math(MathError),
    Parse(ParseIntError),
    Io(io::Error),
    Custom(String),
}

impl fmt::Display for AppError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            AppError::Math(e) => write!(f, "Math error: {}", e),
            AppError::Parse(e) => write!(f, "Parse error: {}", e),
            AppError::Io(e) => write!(f, "IO error: {}", e),
            AppError::Custom(msg) => write!(f, "Application error: {}", msg),
        }
    }
}

impl std::error::Error for AppError {
    fn source(&self) -> Option<&(dyn std::error::Error + 'static)> {
        match self {
            AppError::Math(e) => Some(e),
            AppError::Parse(e) => Some(e),
            AppError::Io(e) => Some(e),
            AppError::Custom(_) => None,
        }
    }
}

// Automatic conversion from specific error types
impl From<MathError> for AppError {
    fn from(error: MathError) -> Self {
        AppError::Math(error)
    }
}

impl From<ParseIntError> for AppError {
    fn from(error: ParseIntError) -> Self {
        AppError::Parse(error)
    }
}

impl From<io::Error> for AppError {
    fn from(error: io::Error) -> Self {
        AppError::Io(error)
    }
}

// Function that can produce multiple types of errors
fn process_input_string(input: &str) -> Result<f64, AppError> {
    // This can produce a ParseIntError
    let number: i32 = input.parse()?;  // Automatically converted to AppError
    
    // Validate range
    if number < 0 {
        return Err(AppError::Custom("Number must be non-negative".to_string()));
    }
    
    if number > 1000 {
        return Err(AppError::Custom("Number too large (max 1000)".to_string()));
    }
    
    // This can produce a MathError
    let result = safe_sqrt(number as f64)?;  // Automatically converted to AppError
    
    Ok(result)
}

// Batch processing with detailed error reporting
fn batch_process(inputs: Vec<&str>) -> Result<Vec<f64>, Vec<(usize, AppError)>> {
    let mut results = Vec::new();
    let mut errors = Vec::new();
    
    for (index, input) in inputs.iter().enumerate() {
        match process_input_string(input) {
            Ok(result) => results.push(result),
            Err(e) => errors.push((index, e)),
        }
    }
    
    if errors.is_empty() {
        Ok(results)
    } else {
        Err(errors)
    }
}

fn multiple_error_types_demo() {
    println!("=== Multiple Error Types Demo ===");
    
    let test_inputs = vec![
        "25",      // Valid
        "abc",     // Parse error
        "-5",      // Custom error (negative)
        "1500",    // Custom error (too large)
        "100",     // Valid
        "0",       // Valid (edge case)
    ];
    
    println!("Processing individual inputs:");
    for input in &test_inputs {
        match process_input_string(input) {
            Ok(result) => println!("✅ '{}' -> √{} = {:.2}", input, input, result),
            Err(e) => println!("❌ '{}' -> {}", input, e),
        }
    }
    
    println!("\nBatch processing:");
    match batch_process(test_inputs) {
        Ok(results) => {
            println!("✅ All inputs processed successfully:");
            for (i, result) in results.iter().enumerate() {
                println!("   Result {}: {:.2}", i + 1, result);
            }
        },
        Err(errors) => {
            println!("❌ Batch processing failed with {} errors:", errors.len());
            for (index, error) in errors {
                println!("   Input {}: {}", index + 1, error);
            }
        },
    }
}

multiple_error_types_demo();

### Error Handling Strategies

In [ ]:
// Different strategies for handling errors

// Strategy 1: Fail fast - propagate errors immediately
fn fail_fast_strategy(inputs: Vec<&str>) -> Result<f64, AppError> {
    let mut sum = 0.0;
    
    for input in inputs {
        let value = process_input_string(input)?;  // Fail on first error
        sum += value;
    }
    
    Ok(sum)
}

// Strategy 2: Collect all errors
fn collect_errors_strategy(inputs: Vec<&str>) -> Result<f64, Vec<AppError>> {
    let mut sum = 0.0;
    let mut errors = Vec::new();
    
    for input in inputs {
        match process_input_string(input) {
            Ok(value) => sum += value,
            Err(e) => errors.push(e),
        }
    }
    
    if errors.is_empty() {
        Ok(sum)
    } else {
        Err(errors)
    }
}

// Strategy 3: Skip errors with logging
fn skip_errors_strategy(inputs: Vec<&str>) -> (f64, Vec<String>) {
    let mut sum = 0.0;
    let mut warnings = Vec::new();
    
    for input in inputs {
        match process_input_string(input) {
            Ok(value) => sum += value,
            Err(e) => {
                warnings.push(format!("Skipped '{}': {}", input, e));
            },
        }
    }
    
    (sum, warnings)
}

// Strategy 4: Retry with fallback
fn retry_with_fallback(input: &str, fallback: f64) -> f64 {
    // Try primary processing
    if let Ok(result) = process_input_string(input) {
        return result;
    }
    
    // Try alternative parsing (as float directly)
    if let Ok(number) = input.parse::<f64>() {
        if number >= 0.0 {
            if let Ok(result) = safe_sqrt(number) {
                return result;
            }
        }
    }
    
    // Use fallback
    fallback
}

fn error_strategies_demo() {
    println!("=== Error Handling Strategies Demo ===");
    
    let test_data = vec!["4", "abc", "16", "-5", "25"];
    
    println!("Test data: {:?}\n", test_data);
    
    // Strategy 1: Fail fast
    println!("Strategy 1 - Fail Fast:");
    match fail_fast_strategy(test_data.clone()) {
        Ok(sum) => println!("✅ Sum: {:.2}", sum),
        Err(e) => println!("❌ Failed: {}", e),
    }
    
    // Strategy 2: Collect all errors
    println!("\nStrategy 2 - Collect All Errors:");
    match collect_errors_strategy(test_data.clone()) {
        Ok(sum) => println!("✅ Sum: {:.2}", sum),
        Err(errors) => {
            println!("❌ {} errors occurred:", errors.len());
            for (i, error) in errors.iter().enumerate() {
                println!("   {}: {}", i + 1, error);
            }
        },
    }
    
    // Strategy 3: Skip errors
    println!("\nStrategy 3 - Skip Errors:");
    let (sum, warnings) = skip_errors_strategy(test_data.clone());
    println!("✅ Sum of valid inputs: {:.2}", sum);
    if !warnings.is_empty() {
        println!("⚠️  Warnings:");
        for warning in warnings {
            println!("   {}", warning);
        }
    }
    
    // Strategy 4: Retry with fallback
    println!("\nStrategy 4 - Retry with Fallback:");
    let fallback_value = 1.0;
    let mut sum = 0.0;
    
    for input in &test_data {
        let result = retry_with_fallback(input, fallback_value);
        println!("  '{}' -> {:.2}", input, result);
        sum += result;
    }
    
    println!("✅ Total sum with fallbacks: {:.2}", sum);
}

error_strategies_demo();

---

## 🎯 Guided Practice

### Exercise: File Processing with Error Handling

Create a robust file processing system with comprehensive error handling.

In [ ]:
// TODO: Complete the file processing system

#[derive(Debug)]
enum FileProcessingError {
    InvalidFormat(String),
    EmptyFile,
    ProcessingError(String),
    ValidationError(String),
}

impl fmt::Display for FileProcessingError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            FileProcessingError::InvalidFormat(msg) => write!(f, "Invalid format: {}", msg),
            FileProcessingError::EmptyFile => write!(f, "File is empty"),
            FileProcessingError::ProcessingError(msg) => write!(f, "Processing error: {}", msg),
            FileProcessingError::ValidationError(msg) => write!(f, "Validation error: {}", msg),
        }
    }
}

impl std::error::Error for FileProcessingError {}

#[derive(Debug, Clone)]
struct DataRecord {
    id: u32,
    name: String,
    value: f64,
}

impl DataRecord {
    // TODO: Parse a CSV line into a DataRecord
    fn from_csv_line(line: &str, line_number: usize) -> Result<Self, FileProcessingError> {
        let parts: Vec<&str> = line.split(',').collect();
        
        if parts.len() != 3 {
            return Err(FileProcessingError::InvalidFormat(
                format!("Line {}: Expected 3 fields, found {}", line_number, parts.len())
            ));
        }
        
        let id = parts[0].trim().parse::<u32>()
            .map_err(|_| FileProcessingError::InvalidFormat(
                format!("Line {}: Invalid ID '{}'", line_number, parts[0])
            ))?;
        
        let name = parts[1].trim().to_string();
        if name.is_empty() {
            return Err(FileProcessingError::ValidationError(
                format!("Line {}: Name cannot be empty", line_number)
            ));
        }
        
        let value = parts[2].trim().parse::<f64>()
            .map_err(|_| FileProcessingError::InvalidFormat(
                format!("Line {}: Invalid value '{}'", line_number, parts[2])
            ))?;
        
        if value < 0.0 {
            return Err(FileProcessingError::ValidationError(
                format!("Line {}: Value must be non-negative, got {}", line_number, value)
            ));
        }
        
        Ok(DataRecord { id, name, value })
    }
    
    // TODO: Validate the record
    fn validate(&self) -> Result<(), FileProcessingError> {
        if self.name.len() < 2 {
            return Err(FileProcessingError::ValidationError(
                format!("Name '{}' is too short (minimum 2 characters)", self.name)
            ));
        }
        
        if self.value > 1000.0 {
            return Err(FileProcessingError::ValidationError(
                format!("Value {} exceeds maximum allowed (1000.0)", self.value)
            ));
        }
        
        Ok(())
    }
}

// TODO: Process CSV data with comprehensive error handling
fn process_csv_data(csv_content: &str) -> Result<Vec<DataRecord>, Vec<FileProcessingError>> {
    let lines: Vec<&str> = csv_content.lines().collect();
    
    if lines.is_empty() {
        return Err(vec![FileProcessingError::EmptyFile]);
    }
    
    let mut records = Vec::new();
    let mut errors = Vec::new();
    
    // Skip header if present
    let data_lines = if lines[0].to_lowercase().contains("id") {
        &lines[1..]
    } else {
        &lines[..]
    };
    
    for (index, line) in data_lines.iter().enumerate() {
        let line_number = index + 2; // Account for header and 1-based indexing
        
        if line.trim().is_empty() {
            continue; // Skip empty lines
        }
        
        match DataRecord::from_csv_line(line, line_number) {
            Ok(record) => {
                match record.validate() {
                    Ok(()) => records.push(record),
                    Err(e) => errors.push(e),
                }
            },
            Err(e) => errors.push(e),
        }
    }
    
    if errors.is_empty() {
        Ok(records)
    } else {
        Err(errors)
    }
}

fn file_processing_demo() {
    println!("=== File Processing Demo ===");
    
    // Sample CSV data with various issues
    let csv_data = r#"ID,Name,Value
1,Alice,25.5
2,Bob,30.0
abc,Charlie,15.0
4,,20.0
5,David,-5.0
6,Eve,1500.0
7,Frank,45.5
8,Grace
9,Henry,35.0
10,A,10.0"#;
    
    println!("Processing CSV data:");
    println!("{}", csv_data);
    println!("\nResults:");
    
    match process_csv_data(csv_data) {
        Ok(records) => {
            println!("✅ Successfully processed {} records:", records.len());
            for record in records {
                println!("   ID: {}, Name: {}, Value: {:.2}", 
                        record.id, record.name, record.value);
            }
        },
        Err(errors) => {
            println!("❌ Processing failed with {} errors:", errors.len());
            for (i, error) in errors.iter().enumerate() {
                println!("   {}: {}", i + 1, error);
            }
        },
    }
    
    // Test with empty data
    println!("\n=== Testing Empty Data ===");
    match process_csv_data("") {
        Ok(_) => println!("✅ Empty data processed successfully"),
        Err(errors) => {
            for error in errors {
                println!("❌ {}", error);
            }
        },
    }
}

file_processing_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. When should you use `panic!` vs `Result<T, E>`?
2. How do you implement the `Display` trait for custom error types?
3. What does the `?` operator do and when can you use it?
4. How do you convert between different error types?
5. What's the difference between `unwrap()`, `expect()`, and pattern matching?
6. How do you handle multiple error types in a single function?
7. What are the benefits of implementing `std::error::Error` for custom types?
8. How do you collect all errors instead of failing on the first one?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How does Rust's error handling approach compare to exceptions in other languages?**
2. **What are the trade-offs between different error handling strategies?**
3. **How do you decide what should be a recoverable vs unrecoverable error?**
4. **What makes error messages helpful for users and developers?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Generics Fundamentals

In our next lesson, you'll learn about:
- Generic functions and structs for code reuse
- Type parameters and constraints
- Monomorphization and zero-cost abstractions
- Generic error types and Result patterns

### How This Connects
Error handling is fundamental to understanding:
- How `Result<T, E>` and `Option<T>` are generic types
- Generic error handling patterns
- Trait bounds for error conversion
- Library design with flexible error types

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Design custom error types using enums and structs?
- [ ] Use the `?` operator effectively for error propagation?
- [ ] Handle multiple error types in a single function?
- [ ] Choose appropriate error handling strategies for different scenarios?
- [ ] Implement `Display` and `Error` traits for custom types?
- [ ] Convert between different error types using `From` trait?
- [ ] Write robust error handling code for real applications?

If you checked all boxes, excellent! You've mastered Rust's powerful error handling system.

---

**🎉 Exceptional Progress!** You now know how to build robust, error-resilient Rust applications that handle failures gracefully!